## Prompt Pair Generation

This notebook is to generate the semantically equivalent, perturbed prompt pairs according to the 20 categories of the 5 logical laws outlined in the MetaLogic research paper, as well as 4 semantic dimensions to test: object presence/deletion, horizontal spatial reasoning, vertical spatial reasoning, and attribute binding.

In [ ]:
import argparse
import csv
import random
from typing import Tuple, Dict

In [ ]:
LOGICAL_LAWS = [
    {"id": "comm",    "name": "Commutative"},
    {"id": "assoc",   "name": "Associative"},
    {"id": "dist",    "name": "Distributive"},
    {"id": "compl",   "name": "Complement"},
    {"id": "demorgan","name": "DeMorgan"},
]

SEMANTIC_DIMS = [
    {"id": "conj",   "name": "Conjunctive"},
    {"id": "horiz",  "name": "Horizontal"},
    {"id": "vert",   "name": "Vertical"},
    {"id": "attr",   "name": "Attributes"},
]

# Simple vocab so scenes stay easy for VQA
OBJECTS = ["cat", "dog", "apple", "banana"]
COLORS = ["red", "blue", "green", "yellow"]
SIZES = ["small", "large"]
SURFACES = ["a wooden table", "a plain white surface", "a dark metal floor"]


def choice_except(seq, avoid):
    """Pick an element from seq that is not 'avoid'."""
    candidates = [x for x in seq if x != avoid]
    return random.choice(candidates) if candidates else avoid


def generate_commutative_conjunctive() -> Tuple[str, str]:
    # A ∧ B ≡ B ∧ A (object presence)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} and a {color_b} {obj_b} on {surface}"
    prompt_b = f"a {color_b} {obj_b} and a {color_a} {obj_a} on {surface}"
    return prompt_a, prompt_b


def generate_commutative_horizontal() -> Tuple[str, str]:
    # A left of B ≡ B right of A
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} to the left of a {color_b} {obj_b} on {surface}"
    prompt_b = f"a {color_b} {obj_b} to the right of a {color_a} {obj_a} on {surface}"
    return prompt_a, prompt_b


def generate_commutative_vertical() -> Tuple[str, str]:
    # A above B ≡ B below A
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} above a {color_b} {obj_b} on {surface}"
    prompt_b = f"a {color_b} {obj_b} below a {color_a} {obj_a} on {surface}"
    return prompt_a, prompt_b


def generate_commutative_attributes() -> Tuple[str, str]:
    # (red ∧ shiny) cube ≡ (shiny ∧ red) cube
    color = random.choice(COLORS)
    size = random.choice(SIZES)
    obj = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    # Just swap the attribute order
    prompt_a = f"a {size} {color} {obj} on {surface}"
    prompt_b = f"a {color} {size} {obj} on {surface}"
    return prompt_a, prompt_b


def generate_associative_conjunctive() -> Tuple[str, str]:
    # (A ∧ B) ∧ C ≡ A ∧ (B ∧ C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} and a {color_b} {obj_b}, "
        f"with a {color_c} {obj_c} next to them on {surface}"
    )
    prompt_b = (
        f"a {color_a} {obj_a}, with a {color_b} {obj_b} and a {color_c} {obj_c} next to it on {surface}"
    )
    return prompt_a, prompt_b


def generate_associative_horizontal() -> Tuple[str, str]:
    # (A left of B) left of C ≡ A left of (B left of C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} to the left of a {color_b} {obj_b}, "
        f"which is to the left of a {color_c} {obj_c} on {surface}"
    )
    prompt_b = (
        f"a {color_a} {obj_a} to the left of a {color_b} {obj_b} and a {color_c} {obj_c} "
        f"lined up on {surface}"
    )
    return prompt_a, prompt_b


def generate_associative_vertical() -> Tuple[str, str]:
    # (A above B) above C ≡ A above (B above C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} above a {color_b} {obj_b}, which is above a {color_c} {obj_c} on {surface}"
    )
    prompt_b = (
        f"a vertical stack with a {color_a} {obj_a} on top, then a {color_b} {obj_b}, "
        f"and a {color_c} {obj_c} on {surface}"
    )
    return prompt_a, prompt_b


def generate_associative_attributes() -> Tuple[str, str]:
    # (small ∧ red) ∧ shiny ≡ small ∧ (red ∧ shiny)
    size = random.choice(SIZES)
    color = random.choice(COLORS)
    obj = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    prompt_a = f"a {size}, {color}, shiny {obj} on {surface}"
    prompt_b = f"a shiny {size}, {color} {obj} on {surface}"
    return prompt_a, prompt_b


def generate_distributive_conjunctive() -> Tuple[str, str]:
    # A ∧ (B ∨ C) ≡ (A ∧ B) ∨ (A ∧ C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} and either a {color_b} {obj_b} or a {color_c} {obj_c} on {surface}"
    )
    prompt_b = (
        f"either a {color_a} {obj_a} with a {color_b} {obj_b} or "
        f"a {color_a} {obj_a} with a {color_c} {obj_c} on {surface}"
    )
    return prompt_a, prompt_b


def generate_distributive_horizontal() -> Tuple[str, str]:
    # A left of (B or C) ≡ (A left of B) or (A left of C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} to the left of either a {color_b} {obj_b} or a {color_c} {obj_c} on {surface}"
    )
    prompt_b = (
        f"either a {color_a} {obj_a} to the left of a {color_b} {obj_b} or "
        f"a {color_a} {obj_a} to the left of a {color_c} {obj_c} on {surface}"
    )
    return prompt_a, prompt_b


def generate_distributive_vertical() -> Tuple[str, str]:
    # A above (B or C) ≡ (A above B) or (A above C)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    color_c = choice_except(COLORS, color_b)
    obj_c = choice_except(OBJECTS, obj_b)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color_a} {obj_a} above either a {color_b} {obj_b} or a {color_c} {obj_c} on {surface}"
    )
    prompt_b = (
        f"either a {color_a} {obj_a} above a {color_b} {obj_b} or "
        f"a {color_a} {obj_a} above a {color_c} {obj_c} on {surface}"
    )
    return prompt_a, prompt_b


def generate_distributive_attributes() -> Tuple[str, str]:
    # red and (small or large) ≡ (red and small) or (red and large)
    color = random.choice(COLORS)
    size1 = random.choice(SIZES)
    size2 = choice_except(SIZES, size1)
    obj = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"a {color} {obj} that is either {size1} or {size2} on {surface}"
    )
    prompt_b = (
        f"either a {size1}, {color} {obj} or a {size2}, {color} {obj} on {surface}"
    )
    return prompt_a, prompt_b


def generate_complement_conjunctive() -> Tuple[str, str]:
    # A vs ¬A (object presence)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} on {surface}"
    prompt_b = f"{surface} with no {color_a} {obj_a} visible"
    return prompt_a, prompt_b


def generate_complement_horizontal() -> Tuple[str, str]:
    # A: object on left vs ¬A: not on left (explicit “not”)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} to the left of a {color_b} {obj_b} on {surface}"
    prompt_b = (
        f"{surface} with a {color_b} {obj_b}, but no {color_a} {obj_a} to its left"
    )
    return prompt_a, prompt_b


def generate_complement_vertical() -> Tuple[str, str]:
    # A vs ¬A (above relation)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color_a} {obj_a} above a {color_b} {obj_b} on {surface}"
    prompt_b = (
        f"{surface} with a {color_b} {obj_b}, but no {color_a} {obj_a} above it"
    )
    return prompt_a, prompt_b


def generate_complement_attributes() -> Tuple[str, str]:
    # A vs ¬A in attributes (e.g., red vs not red)
    color = random.choice(COLORS)
    other_color = choice_except(COLORS, color)
    obj = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    prompt_a = f"a {color} {obj} on {surface}"
    prompt_b = f"{surface} with a {obj} that is not {color}, such as a {other_color} {obj}"
    return prompt_a, prompt_b


def generate_demorgan_conjunctive() -> Tuple[str, str]:
    # ¬(A ∧ B) ≡ ¬A ∨ ¬B
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"{surface} where it is not the case that both a {color_a} {obj_a} "
        f"and a {color_b} {obj_b} are present"
    )
    prompt_b = (
        f"{surface} with either no {color_a} {obj_a} or no {color_b} {obj_b} present"
    )
    return prompt_a, prompt_b


def generate_demorgan_horizontal() -> Tuple[str, str]:
    # ¬(A left of B) ≡ ¬A ∨ ¬B (in that relation)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"{surface} where it is not the case that a {color_a} {obj_a} is to the left "
        f"of a {color_b} {obj_b}"
    )
    prompt_b = (
        f"{surface} with either no {color_a} {obj_a} or no {color_b} {obj_b} in a left-right arrangement"
    )
    return prompt_a, prompt_b


def generate_demorgan_vertical() -> Tuple[str, str]:
    # ¬(A above B) ≡ ¬A ∨ ¬B (in that relation)
    color_a = random.choice(COLORS)
    obj_a = random.choice(OBJECTS)
    color_b = choice_except(COLORS, color_a)
    obj_b = choice_except(OBJECTS, obj_a)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"{surface} where it is not the case that a {color_a} {obj_a} is above "
        f"a {color_b} {obj_b}"
    )
    prompt_b = (
        f"{surface} with either no {color_a} {obj_a} or no {color_b} {obj_b} in an above-below arrangement"
    )
    return prompt_a, prompt_b


def generate_demorgan_attributes() -> Tuple[str, str]:
    # ¬(red ∧ small) ≡ ¬red ∨ ¬small
    color = random.choice(COLORS)
    other_color = choice_except(COLORS, color)
    size = random.choice(SIZES)
    other_size = choice_except(SIZES, size)
    obj = random.choice(OBJECTS)
    surface = random.choice(SURFACES)

    prompt_a = (
        f"{surface} where it is not the case that there is a {size}, {color} {obj}"
    )
    prompt_b = (
        f"{surface} with a {obj} that is either not {color} or not {size}, "
        f"such as a {other_size}, {other_color} {obj}"
    )
    return prompt_a, prompt_b




In [ ]:
# Map (law_id, semantic_id) -> generator function
GEN_FUNCS: Dict[Tuple[str, str], callable] = {
    ("comm", "conj"):   generate_commutative_conjunctive,
    ("comm", "horiz"):  generate_commutative_horizontal,
    ("comm", "vert"):   generate_commutative_vertical,
    ("comm", "attr"):   generate_commutative_attributes,

    ("assoc", "conj"):  generate_associative_conjunctive,
    ("assoc", "horiz"): generate_associative_horizontal,
    ("assoc", "vert"):  generate_associative_vertical,
    ("assoc", "attr"):  generate_associative_attributes,

    ("dist", "conj"):   generate_distributive_conjunctive,
    ("dist", "horiz"):  generate_distributive_horizontal,
    ("dist", "vert"):   generate_distributive_vertical,
    ("dist", "attr"):   generate_distributive_attributes,

    ("compl", "conj"):  generate_complement_conjunctive,
    ("compl", "horiz"): generate_complement_horizontal,
    ("compl", "vert"):  generate_complement_vertical,
    ("compl", "attr"):  generate_complement_attributes,

    ("demorgan", "conj"):  generate_demorgan_conjunctive,
    ("demorgan", "horiz"): generate_demorgan_horizontal,
    ("demorgan", "vert"):  generate_demorgan_vertical,
    ("demorgan", "attr"):  generate_demorgan_attributes,
}


def parse_args():
    p = argparse.ArgumentParser(description="Generate MetaLogic prompt pairs CSV")
    p.add_argument("--out", type=str, default="prompts_metalogic.csv",
                   help="Output CSV file path")
    p.add_argument("--pairs-per-category", type=int, default=5,
                   help="How many prompt pairs to generate for each of the 20 categories")
    p.add_argument("--seed", type=int, default=42,
                   help="Random seed for reproducibility")
    return p.parse_args([])


def main():
    args = parse_args()
    random.seed(args.seed)

    rows = []
    category_counter = 0

    for law in LOGICAL_LAWS:
        for sem in SEMANTIC_DIMS:
            category_counter += 1
            cat_id = category_counter
            law_id = law["id"]
            law_name = law["name"]
            sem_id = sem["id"]
            sem_name = sem["name"]

            gen_fn = GEN_FUNCS[(law_id, sem_id)]

            for i in range(1, args.pairs_per_category + 1):
                prompt_a, prompt_b = gen_fn()
                pair_id = f"{law_id}_{sem_id}_{i:03d}"
                rows.append({
                    "pair_id": pair_id,
                    "category_id": cat_id,
                    "logical_law": law_name,
                    "semantic_dimension": sem_name,
                    "prompt_A": prompt_a,
                    "prompt_B": prompt_b,
                })

    fieldnames = [
        "pair_id",
        "category_id",
        "logical_law",
        "semantic_dimension",
        "prompt_A",
        "prompt_B",
    ]

    with open(args.out, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

    print(f"Wrote {len(rows)} prompt pairs to {args.out}")
    print(f"Categories: {category_counter}, pairs per category: {args.pairs_per_category}")


if __name__ == "__main__":
    main()

Wrote 100 prompt pairs to prompts_metalogic.csv
Categories: 20, pairs per category: 5
